In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output
import base64

# Configure OS routines
import os

# Data + plotting
import pandas as pd

# Import your CRUD Python module
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

USERNAME = "aacuser"
PASSWORD = "SNHU1234"
HOST     = "nv-desktop-services.apporto.com"
PORT     = 32818
DB       = "AAC"
COL      = "animals"

shelter = AnimalShelter(USERNAME, PASSWORD, HOST, PORT, DB, COL)

# Initial dataset
df = pd.DataFrame.from_records(shelter.read({}))
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

# Ensure columns exist
for col in ["location_lat", "location_long", "breed", "name"]:
    if col not in df.columns:
        df[col] = None

###########################
# Logo setup
###########################
image_filename = "Grazioso Salvare Logo.png"
encoded_image = base64.b64encode(open(image_filename, "rb").read())

###########################
# Dashboard Layout / View
###########################
app = JupyterDash(__name__)

app.layout = html.Div([
    # Header
    html.Div(
        style={"display": "flex", "alignItems": "center", "gap": "16px"},
        children=[
            html.Img(src="data:image/png;base64,{}".format(encoded_image.decode()), height="80px"),
            html.Div([
                html.H2("Grazioso Salvare Dashboard"),
                html.P("Developed by Aqbah Aamir", style={"color": "gray"})
            ])
        ]
    ),
    html.Hr(),

    # Filter options
    html.Div([
        html.Label("Select Rescue Type:", style={"fontWeight": "bold"}),
        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "Water Rescue", "value": "Water Rescue"},
                {"label": "Mountain/Wilderness", "value": "Mountain/Wilderness"},
                {"label": "Disaster/Tracking", "value": "Disaster/Tracking"},
                {"label": "Reset", "value": "Reset"}
            ],
            value="Reset",
            inline=True
        )
    ]),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict("records"),
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_current=0,
        page_size=10,
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX": "auto"},
        style_cell={"fontSize": 12, "padding": "6px"},
        style_header={"fontWeight": "bold"},
    ),

    html.Br(),
    html.Hr(),

    # Charts side by side
    html.Div(className="row", style={"display": "flex"}, children=[
        html.Div(id="graph-id", className="col s12 m6"),
        html.Div(id="map-id", className="col s12 m6")
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filter mappings
MAX_AGE_WEEKS = 156
FILTERS = {
    "Water Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "age_upon_outcome_in_weeks": {"$lte": MAX_AGE_WEEKS}
    },
    "Mountain/Wilderness": {
        "animal_type": "Dog",
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                          "Siberian Husky", "Rottweiler"]},
        "age_upon_outcome_in_weeks": {"$lte": MAX_AGE_WEEKS}
    },
    "Disaster/Tracking": {
        "animal_type": "Dog",
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                          "Bloodhound", "Rottweiler"]},
        "age_upon_outcome_in_weeks": {"$lte": MAX_AGE_WEEKS}
    },
    "Reset": {"animal_type": "Dog"}
}

def run_query(choice):
    q = FILTERS.get(choice, FILTERS["Reset"])
    results = shelter.read(q)
    dff = pd.DataFrame.from_records(results)
    if "_id" in dff.columns:
        dff.drop(columns=["_id"], inplace=True)
    for col in ["location_lat", "location_long", "breed", "name"]:
        if col not in dff.columns:
            dff[col] = None
    return dff

# Update DataTable when filter changes
@app.callback(
    Output("datatable-id", "data"),
    Output("datatable-id", "columns"),
    Output("datatable-id", "selected_rows"),
    Input("filter-type", "value")
)
def update_dashboard(filter_type):
    dff = run_query(filter_type)
    cols = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in dff.columns]
    return dff.to_dict("records"), cols, [0] if len(dff) > 0 else []

# Update Pie Chart
@app.callback(
    Output("graph-id", "children"),
    Input("datatable-id", "data")
)
def update_graphs(viewData):
    dff = pd.DataFrame(viewData)
    if dff.empty or "breed" not in dff.columns:
        return dcc.Graph(figure=px.pie(title="No data"))
    top = (
        dff["breed"].fillna("Unknown").value_counts().nlargest(10).reset_index()
        .rename(columns={"index": "breed", "breed": "count"})
    )
    fig = px.pie(top, names="breed", values="count", hole=0.3,
                 title="Top Breeds in Current Filter")
    return dcc.Graph(figure=fig)

# Highlight selected column
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    Input("datatable-id", "selected_columns")
)
def update_styles(selected_columns):
    return [{
        "if": {"column_id": i},
        "background_color": "#D2F3FF"
    } for i in selected_columns]

# Update Map
@app.callback(
    Output("map-id", "children"),
    Input("datatable-id", "data"),
    Input("datatable-id", "selected_rows")
)
def update_map(viewData, index):  
    # if no data in table yet, stop
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data to display on map.")]

    dff = pd.DataFrame.from_dict(viewData)

    # default to first row if no row is selected
    row = index[0] if index else 0
    
    try:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row,13], dff.iloc[row,14]],
                    children=[
                        dl.Tooltip(dff.iloc[row,4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row,9])
                        ])
                    ]
                )
            ])
        ]
    except Exception as e:
        return [html.Div(f"Error displaying map: {str(e)}")]

app.run_server(debug=True, mode="inline")


Successfully connected to MongoDB!


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: 'NoneType' object is not iterable

